# Baseline Model based on U-Net

In [1]:
import os
os.chdir("..")

## Training

In [2]:
from baseline.trainer import Trainer

trainer = Trainer()

Segmentation Models: using `tf.keras` framework.


In [ ]:
%time history = trainer.train_from_simple_dataloader(dataset_size=10)

In [ ]:
plt.figure(figsize=(30, 5))
plt.subplot(121)
plt.plot(history.history['iou_score'])
plt.plot(history.history['val_iou_score'])
plt.title('Model iou_score')
plt.ylabel('iou_score')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(122)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

## Prediction

In [ ]:
import segmentation_models as sm
from constants import TEST_DIR
from baseline.dataloader import SimpleDataLoader
from baseline.data_augmentation import DataAugmentation
from baseline.helper import Visualisation

simple_data_loader = SimpleDataLoader(
    images_path=os.path.join(TEST_DIR, "images"),
    preprocessing=DataAugmentation().get_preprocessing(sm.get_preprocessing(Trainer.BACKBONE)),
    size=10
)

test_images = simple_data_loader.get_images()

In [ ]:
import numpy as np

test_image = test_images[0]

print(test_image.shape)
test_image = np.expand_dims(test_image, axis=0)  # add batch dimension
print(test_image.shape)

Visualisation().show(image=test_image.squeeze())

In [ ]:
model = trainer.get_model()
model.summary()

In [ ]:
predected_mask = model.predict(test_image)
print(predicted_mask)

In [ ]:
for test_image in simple_data_loader.get_images()[:3]:
    print(test_image)
    predicted_mask = trainer.get_model().predict(test_image).round()
    print(predicted_mask)
    
    Visualisation().show(
        image=test_image.squeeze(),
        predicted_mask=predicted_mask[..., 0].squeeze()
    )